### **Welcome abroad!**
`boson_ladder` is _fully_ based on `SymPy`. However, there is no need to specifically import anything from `SymPy` to use the features of this package, except if you want to use `sympy.Symbol` as variables.

In [1]:
import boson_ladder as bl
from sympy import *

---

### **The bosonic ladder operator objects**

In `SymPy`, the ladder operators are available as `sympy.physics.secondquant.AnnihilateBoson` and `sympy.physics.secondquant.CreateBoson`. In this package, the recommended way to get these objects is by calling [`boson_ladder.ops`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/utils/operators.py?plain=1#L19):

In [19]:
b_1, bd_1 = bl.ops(1)
b_2, bd_2 = bl.ops(2)

b_1

AnnihilateBoson(1)

In [3]:
bd_1

CreateBoson(1)

There are several advantages of calling this function over calling the `SymPy` constructors: 
-   You do not need to manually import `sympy.physics.secondquant` which requires a separate import. 
-   You also only need to call one thing instead of two. 
-   As shown above, you can input any nonzero integer as a subscript without the constructor raising an error when printing the latex output (this may just be a bug):

In [4]:
from sympy.physics.secondquant import AnnihilateBoson
try:
    latex(AnnihilateBoson(1))
except:
    print("An exception is raised!")

An exception is raised!


If you are working with single-mode systems, you can remove the subscript by specifying no input:

In [5]:
b, bd = bl.ops()
b

AnnihilateBoson()

In [6]:
bd

CreateBoson()

which cannot be done in the current version of `SymPy`:

In [7]:
from sympy.physics.secondquant import AnnihilateBoson
try:
    AnnihilateBoson()
except:
    print("Whoa, there!")

Whoa, there!


Other than the construction process, the outputs of `ops` are really the `SymPy` objects:

In [8]:
isinstance(b, AnnihilateBoson)

True

---

### **Normal ordering**

Normal ordering means using the commutation relations to obtain an expression where all creation operators $b_j^\dagger$ are positioned to the left of all annihilation operators $b_j$. Other than making the expressions nicer to see, normal-ordered operators behave nicer in quantum mechanics (see the [optical equivalence theorem](https://en.wikipedia.org/wiki/Optical_equivalence_theorem)). 

The function [`normal_ordering`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/do_commutator.py#L170) does normal ordering like how you probably would: it looks for a creation operator to the right of an annihilation operator, then apply the commutation relations to swap their places. This is done recursively until we have a normal-ordered expression.

In [ ]:
bl.normal_ordering(b_1 * bd_1**2)

2*CreateBoson() + CreateBoson()**2*AnnihilateBoson()

Many-mode input:

In [20]:
bl.normal_ordering(b_2** 3 * b_1 * bd_2**2 * b_1**2)

6*AnnihilateBoson(2)*AnnihilateBoson(1)**3 + 6*CreateBoson(2)*AnnihilateBoson(2)**2*AnnihilateBoson(1)**3 + CreateBoson(2)**2*AnnihilateBoson(2)**3*AnnihilateBoson(1)**3

Polynomial input:

In [21]:
bl.normal_ordering(b_1**2 * bd_2**4 \
                    + 5 * b_2**2 * bd_1**5 * b_1 \
                    + b_2 )

AnnihilateBoson(2) + 5*CreateBoson(1)**5*AnnihilateBoson(2)**2*AnnihilateBoson(1) + CreateBoson(2)**4*AnnihilateBoson(1)**2

Input with symbols:

In [23]:
x = Symbol("x")

bl.normal_ordering(x * b_1 * x**2 * bd_1**2)

2*x**3*CreateBoson(1) + x**3*CreateBoson(1)**2*AnnihilateBoson(1)

---

### **Evaluating commutators**

**(CAUTION)** Non-`sympy.Number` exponents are treated as unbreakable into smaller constituents. As such, commutators like $\left[\hat{b}_k^{\dagger p},\hat{b} \right]$, where $p$ is a `sympy.Symbol`, do not work with this package.

Commutators are evaluated with the function [`do_commutator`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/do_commutator.py#L170). It works by simplifying the input commutator using the identities:
\begin{align}
[\hat{A}\hat{B},\hat{C}] &= \hat{A}[\hat{B},\hat{C}] + [\hat{A},\hat{C}]\hat{B}
\notag
\\
[\hat{A},\hat{B}\hat{C}] &= [\hat{A},\hat{B}]\hat{C} + \hat{B}[\hat{A},\hat{C}]
\notag
\end{align}
(hence the caution above) until `SymPy` can automatically evaluate the commutator. Any commutator of any two polynomials in the ladder operators can be evaluated this way, though the `for`-loops may make it costly for long polynomials. 

>   Use `do_commutator` where you would use `sympy.physics.secondquant.Commutator`.

Here is a commutator that SymPy does not automatically evaluate:

In [11]:
from sympy.physics.secondquant import Commutator

A = bd*b
B = b

sympy_comm = Commutator(A, B)
sympy_comm

Commutator(CreateBoson()*AnnihilateBoson(),b())

In [12]:
sympy_comm.simplify()

-AnnihilateBoson()*CreateBoson()*AnnihilateBoson() + CreateBoson()*AnnihilateBoson()**2

In [13]:
sympy_comm.doit()

-AnnihilateBoson()*CreateBoson()*AnnihilateBoson() + CreateBoson()*AnnihilateBoson()**2

`do_commutator` helps `SymPy` evaluate the commutator:

In [14]:
bl.do_commutator(A, B)

-AnnihilateBoson()

Let us try a more complex commutator:

In [15]:
A = bd**2 * b**3
B = b**5 * bd

bl.do_commutator(A, B)

-60*AnnihilateBoson()**5 - 60*CreateBoson()*AnnihilateBoson()**6 - 7*CreateBoson()**2*AnnihilateBoson()**7

By default, the input is normal ordered. You can disable this by setting the parameter `normal_order = False`.

In [16]:
bl.do_commutator(A, B, normal_order=False)

-2*AnnihilateBoson()*CreateBoson()*AnnihilateBoson()**6*CreateBoson() - 2*AnnihilateBoson()**2*CreateBoson()*AnnihilateBoson()**5*CreateBoson() - 2*AnnihilateBoson()**3*CreateBoson()*AnnihilateBoson()**4*CreateBoson() - 2*AnnihilateBoson()**4*CreateBoson()*AnnihilateBoson()**3*CreateBoson() + 3*AnnihilateBoson()**5*CreateBoson()**2*AnnihilateBoson()**2 - 2*CreateBoson()*AnnihilateBoson()**7*CreateBoson()

Many-mode inputs:

In [ ]:
A = bd_1*bd_2
B = b_1*b_2

bl.do_commutator(A, B)

-1 - CreateBoson(1)*AnnihilateBoson(1) - CreateBoson(2)*AnnihilateBoson(2)

Polynomial inputs:

In [18]:
A = b_1 + 2*b_2**2
B = bd_1**2 * bd_1 + 2*bd_2*b_2

bl.do_commutator(A, B)

8*AnnihilateBoson(2)**2 + 3*CreateBoson(1)**2

Inputs with symbols:

In [26]:
A = x*b_1 
B = x**(0.5)*bd_1*b

bl.do_commutator(A, B)

x**1.5*AnnihilateBoson()

---

### **Expectation value evolution in the Lindblad Master Equation Framework**

The master equation in the Lindblad form, or the Lindblad master equation (LME) is arguably the simplest extension of the Schr&ouml;dinger equation to open quantum systems (i.e. systems whose interaction with their environments are practically intractable). Instead of the wave function vector $\left|\psi\right\rangle$, we describe the system using density matrices $\rho$, which evolves according to
\begin{align}
    \frac{\mathrm{d}\rho}{\mathrm{d}t} = -i\left[\hat{H},\rho\right] + \sum_j \gamma_j \mathcal{D}\left(\hat{O}_j\right)[\rho]
\notag
\end{align}
where the term with the Hamiltonian $\hat{H}$ describes the closed system dynamics, and the rest describe the open system dynamics. Here, $\gamma_j$ are nonnegative scalars, while 
\begin{align}
    \mathcal{D}\left(\hat{O}_j\right)\left[\rho\right] = \hat{O}_j\rho\hat{O}_j^\dagger - \frac{1}{2}\left\{\hat{O}_j^\dagger\hat{O}_j,\rho\right\} \notag
\end{align}
is the Liouvillian superator in the Lindblad form, or more concisely, the Lindblad dissipator. The operators $\hat{O}_j$ describing the dissipator depends on the process (e.g. $\hat{b}$ for a one-quantum dissipation)&mdash; the scalars $\gamma_j$ can be interpreted as the rate of the processes.

It is usually interesting to compute the evolution of some expectation value. For a system described by the density matrix $\rho$, the expectation value of some quantity represented by the operator $\hat{A}$ is given by $\left\langle A \right\rangle=\mathrm{tr}\left(\rho\hat{A}\right)$. The evolution of $\left\langle A \right\rangle$ is given by

\begin{align}
\frac{\mathrm{d}\left\langle A\right\rangle}{\mathrm{d}t} &= \mathrm{tr}\left(\frac{\mathrm{d}\rho}{\mathrm{d}t}\hat{A}\right) \notag
\\
&= -i\ \mathrm{tr}\left(\left[\hat{H},\rho\right]\hat{A}\right) + \sum_j\gamma_j\mathrm{tr}\left( \mathcal{D}\left(\hat{O}_j\right)\left[\rho\right]\hat{A}\right) \notag
\end{align}

We call the first term on the RHS the "Hamiltonian trace" and the rest the "dissipator traces". The most general form of $\hat{H}$ and $\hat{O}_j$ we consider is a polynomial in the ladder operators. In this case, we have the following identities (which can be straightforwardly derived):

\begin{align}
-i\ \mathrm{tr}\left(\left[\hat{H},\rho\right]\hat{A}\right) &= -i\left\langle\left[\hat{A},\hat{H}\right]\right\rangle \notag
\\ \notag
\\
\mathrm{tr}\left( \mathcal{D}\left(\sum_k\hat{\mathcal{O}}_k\right)\left[\rho\right]\hat{A}\right) &= \sum_k \left\langle \hat{\mathcal{O}}_k^\dagger \left[\hat{A}, \hat{\mathcal{O}}_k\right]\right\rangle +\left\langle \left[\hat{\mathcal{O}}_k^\dagger, \hat{A}\right] \hat{\mathcal{O}}_k \right\rangle\notag
\\
&\quad +\sum_{l>k}\left\langle \hat{\mathcal{O}}_k^\dagger \left[\hat{A},\hat{\mathcal{O}}_l\right]\right\rangle + \left\langle \left[ \hat{\mathcal{O}}_k^\dagger,\hat{A}\right]\hat{\mathcal{O}}_l\right\rangle + \left\langle \hat{\mathcal{O}}_l^\dagger \left[\hat{A},\hat{\mathcal{O}}_k\right]\right\rangle + \left\langle \left[ \hat{\mathcal{O}}_l^\dagger,\hat{A}\right]\hat{\mathcal{O}}_k\right\rangle
\notag
\end{align}
where we have absorbed the multiplying scalars in the polynomial into $\hat{\mathcal{O}}_k$. These identities make the basis for the evaluation of the Hamiltonian trace by [`Hamiltonian_trace`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L21) and the dissipator traces by [`dissipator_trace`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L64)&mdash;check them out! 

The two functions make up the main function in this section: [`LME_expval_evo`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L123). You only need to input the Hamiltonian `H`, the list of dissipators and their process rates `D`, and the operator `A` corresponding to the quantity whose expectation value evolution is of your interest&mdash;the function will do the rest. To be more specific, `D` has the syntax `D = [[gamma_0, O_0], [gamma_0, O_1], [gamma_2, O_2], ...]`.

**Example 1: One-dimensional quantum simple harmonic oscillator**

Let us start simple with the simple harmonic oscillator. We have $\hat{H}=\hbar\omega_0\hat{b}^\dagger\hat{b}$ and no dissipators. The evolution of the expected phase point $\left\langle \hat{b} \right\rangle = \mathrm{tr}\left(\rho \hat{b}\right)$ is given by

In [34]:
hbar, omega_0 = symbols(r"hbar omega_0")
b, bd = bl.ops()

H = hbar*omega_0*bd*b
D = []
A = b

bl.LME_expval_evo(H, D, A)

Eq(Derivative({\left\langle b_{} \right\rangle}, t), -I*hbar*omega_0*{\left\langle b_{} \right\rangle})

What about its energy, $\left\langle \hat{b}^\dagger \hat{b} \right\rangle$? We can just set $\hat{A}=\hat{b}^\dagger \hat{b}$. We have

In [35]:
A = bd*b

bl.LME_expval_evo(H, D, A)

Eq(Derivative({\left\langle {b^\dagger_{}} b_{} \right\rangle}, t), 0)

**Example 2: One-dimensional harmonically-driven Kerr oscillator**